In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from statistics import mode
import re
from xgboost import XGBClassifier

In [2]:
#Load dataset - https://www.kaggle.com/nsharan/h-1b-visa/data 
df = pd.read_csv('/Users/rutu/Desktop/Data Science study materials/PROJECTS /Python Project/h1b_kaggle.csv')

In [4]:
#Understanding data 
df.info()
df.head()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002458 entries, 0 to 3002457
Data columns (total 11 columns):
Unnamed: 0            int64
CASE_STATUS           object
EMPLOYER_NAME         object
SOC_NAME              object
JOB_TITLE             object
FULL_TIME_POSITION    object
PREVAILING_WAGE       float64
YEAR                  float64
WORKSITE              object
lon                   float64
lat                   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 252.0+ MB


,Unnamed: 0,PREVAILING_WAGE,YEAR,lon,lat
count,3.002458e+06,3.002373e+06,3.002445e+06,2.895216e+06,2.895216e+06
mean,1.501230e+06,1.469984e+05,2.013855e+03,-9.213441e+01,3.816054e+01
std,8.667351e+05,5.287609e+06,1.680612e+00,1.965591e+01,4.672835e+00
min,1.000000e+00,0.000000e+00,2.011000e+03,-1.578583e+02,1.343719e+01
25%,7.506152e+05,5.437100e+04,2.012000e+03,-1.119261e+02,3.416536e+01
50%,1.501230e+06,6.502100e+04,2.014000e+03,-8.615862e+01,3.910312e+01
75%,2.251844e+06,8.143200e+04,2.015000e+03,-7.551381e+01,4.088374e+01
max,3.002458e+06,6.997607e+09,2.016000e+03,1.457298e+02,6.483778e+01


In [12]:
list(df)

['CASE_ID',
 'CASE_STATUS',
 'EMPLOYER_NAME',
 'SOC_NAME',
 'JOB_TITLE',
 'FULL_TIME_POSITION',
 'PREVAILING_WAGE',
 'YEAR',
 'WORKSITE',
 'lon',
 'lat']

In [11]:
df.rename( columns={'Unnamed: 0':'CASE_ID'}, inplace=True )

In [14]:
#CASE_STATUS target variable
df['CASE_STATUS'].unique()

array(['CERTIFIED-WITHDRAWN', 'WITHDRAWN', 'CERTIFIED', 'DENIED',
       'REJECTED', 'INVALIDATED',
       'PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED', nan],
      dtype=object)

In [21]:
df['CASE_STATUS'].value_counts()

CERTIFIED    2818282
DENIED         94364
WITHDRAWN      89799
Name: CASE_STATUS, dtype: int64

In [20]:
#Data Preprocessing
import warnings
warnings.filterwarnings("ignore")
df.CASE_STATUS[df['CASE_STATUS']=='REJECTED'] = 'DENIED'
df.CASE_STATUS[df['CASE_STATUS']=='INVALIDATED'] = 'DENIED'
df.CASE_STATUS[df['CASE_STATUS']=='PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED'] = 'DENIED'
df.CASE_STATUS[df['CASE_STATUS']=='CERTIFIED-WITHDRAWN'] = 'CERTIFIED'

In [23]:
##Drop rows with withdrawn
df = df.drop(df[df.CASE_STATUS == 'WITHDRAWN'].index)

count             2912628
unique             234089
top       INFOSYS LIMITED
freq               130317
Name: EMPLOYER_NAME, dtype: object

In [24]:
#Handling missing value 
##Storing non null in df w.r.t. case status
df = df[df['CASE_STATUS'].notnull()]
print(df['CASE_STATUS'].value_counts())

CERTIFIED    2818282
DENIED         94364
Name: CASE_STATUS, dtype: int64


In [25]:
print(94364/(2818282+94364))

0.032398032579311044


In [45]:
##check count of NAN
count_nan = len(df) - df.count()
print(count_nan)

CASE_ID                    0
CASE_STATUS                0
EMPLOYER_NAME              0
SOC_NAME                   0
JOB_TITLE                  0
FULL_TIME_POSITION         0
PREVAILING_WAGE            0
YEAR                       0
WORKSITE                   0
lon                   102844
lat                   102844
dtype: int64


In [29]:
##Filling na in employer name with mode
df['EMPLOYER_NAME'] = df['EMPLOYER_NAME'].fillna(df['EMPLOYER_NAME'].mode()[0])

In [37]:
df.PREVAILING_WAGE.describe()

count    2.912593e+06
mean     1.428910e+05
std      5.282312e+06
min      0.000000e+00
25%      5.441300e+04
50%      6.500000e+04
75%      8.143200e+04
max      6.997607e+09
Name: PREVAILING_WAGE, dtype: float64

In [38]:
df.PREVAILING_WAGE.median()

65000.0

In [39]:
##to check the percentile in wages
print(np.nanpercentile(df.PREVAILING_WAGE,98))
print(np.nanpercentile(df.PREVAILING_WAGE,2))

138611.0
34028.8


In [40]:
## replacing min and max with 2 and 98 percentile
df.loc[df.PREVAILING_WAGE < 34028.8, 'PREVAILING_WAGE']= 34028.8
df.loc[df['PREVAILING_WAGE'] > 138611, 'PREVAILING_WAGE']= 138611
df.PREVAILING_WAGE.fillna(df.PREVAILING_WAGE.mean(), inplace = True)

In [41]:
df.PREVAILING_WAGE.describe()

count    2.912646e+06
mean     6.984460e+04
std      2.284287e+04
min      3.402880e+04
25%      5.441300e+04
50%      6.500000e+04
75%      8.143200e+04
max      1.386110e+05
Name: PREVAILING_WAGE, dtype: float64

In [42]:
## Filling na in JOB_TITLE and FULL_TIME_POSITION with mode
df['JOB_TITLE'] = df['JOB_TITLE'].fillna(df['JOB_TITLE'].mode()[0])
df['FULL_TIME_POSITION'] = df['FULL_TIME_POSITION'].fillna(df['FULL_TIME_POSITION'].mode()[0])
df['SOC_NAME'] = df['SOC_NAME'].fillna(df['SOC_NAME'].mode()[0])

In [46]:
# Dropping lat and lon columns
df = df.drop('lat', axis = 1)
df = df.drop('lon', axis = 1)


In [47]:
df['EMPLOYER_NAME'].value_counts()

INFOSYS LIMITED                                       130335
TATA CONSULTANCY SERVICES LIMITED                      64418
WIPRO LIMITED                                          47497
DELOITTE CONSULTING LLP                                36675
ACCENTURE LLP                                          33316
IBM INDIA PRIVATE LIMITED                              30426
MICROSOFT CORPORATION                                  25128
HCL AMERICA, INC.                                      22608
ERNST & YOUNG U.S. LLP                                 18220
LARSEN & TOUBRO INFOTECH LIMITED                       16906
CAPGEMINI AMERICA INC                                  16458
COGNIZANT TECHNOLOGY SOLUTIONS U.S. CORPORATION        16457
GOOGLE INC.                                            16104
IGATE TECHNOLOGIES INC.                                12263
IBM CORPORATION                                        11877
TECH MAHINDRA (AMERICAS),INC.                          10682
INTEL CORPORATION       

In [48]:
#Feature Creation
df['NEW_EMPLOYER'] = np.nan

In [49]:
df.head(5)

,CASE_ID,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,NEW_EMPLOYER
0,1,CERTIFIED,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",NaN
1,2,CERTIFIED,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,138611.0,2016.0,"PLANO, TEXAS",NaN
2,3,CERTIFIED,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,138611.0,2016.0,"JERSEY CITY, NEW JERSEY",NaN
3,4,CERTIFIED,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,138611.0,2016.0,"DENVER, COLORADO",NaN
5,6,CERTIFIED,BURGER KING CORPORATION,CHIEF EXECUTIVES,"EXECUTIVE V P, GLOBAL DEVELOPMENT AND PRESIDEN...",Y,138611.0,2016.0,"MIAMI, FLORIDA",NaN


In [50]:
##the applications submitted by universities are generaly certified and accepted 
df['EMPLOYER_NAME'] = df['EMPLOYER_NAME'].str.lower()

In [51]:
df.NEW_EMPLOYER[df['EMPLOYER_NAME'].str.contains('university')] = 'university'


In [52]:
df['NEW_EMPLOYER']= df.NEW_EMPLOYER.replace(np.nan, 'non university', regex=True)

In [53]:
# Creating occupation and mapping the values

df['OCCUPATION'] = np.nan
df['SOC_NAME'] = df['SOC_NAME'].str.lower()
df.OCCUPATION[df['SOC_NAME'].str.contains('computer','programmer')] = 'computer occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('software','web developer')] = 'computer occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('database')] = 'computer occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('math','statistic')] = 'Mathematical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('predictive model','stats')] = 'Mathematical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('teacher','linguist')] = 'Education Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('professor','Teach')] = 'Education Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('school principal')] = 'Education Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('medical','doctor')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('physician','dentist')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('Health','Physical Therapists')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('surgeon','nurse')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('psychiatr')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('chemist','physicist')] = 'Advance Sciences'
df.OCCUPATION[df['SOC_NAME'].str.contains('biology','scientist')] = 'Advance Sciences'
df.OCCUPATION[df['SOC_NAME'].str.contains('biologi','clinical research')] = 'Advance Sciences'
df.OCCUPATION[df['SOC_NAME'].str.contains('public relation','manage')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('management','operation')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('chief','plan')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('executive')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('advertis','marketing')] = 'Marketing Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('promotion','market research')] = 'Marketing Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('business','business analyst')] = 'Business Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('business systems analyst')] = 'Business Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('accountant','finance')] = 'Financial Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('financial')] = 'Financial Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('engineer','architect')] = 'Architecture & Engineering'
df.OCCUPATION[df['SOC_NAME'].str.contains('surveyor','carto')] = 'Architecture & Engineering'
df.OCCUPATION[df['SOC_NAME'].str.contains('technician','drafter')] = 'Architecture & Engineering'
df.OCCUPATION[df['SOC_NAME'].str.contains('information security','information tech')] = 'Architecture & Engineering'
df['OCCUPATION']= df.OCCUPATION.replace(np.nan, 'Others', regex=True)


In [54]:
#visa applications majorly depend on State location
## Splitting city and state and capturing state in another variable
df['state'] = df.WORKSITE.str.split('\s+').str[-1]

In [55]:
df.head(5)

,CASE_ID,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,NEW_EMPLOYER,OCCUPATION,state
0,1,CERTIFIED,university of michigan,biochemists and biophysicists,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",university,Advance Sciences,MICHIGAN
1,2,CERTIFIED,"goodman networks, inc.",chief executives,CHIEF OPERATING OFFICER,Y,138611.0,2016.0,"PLANO, TEXAS",non university,Management Occupation,TEXAS
2,3,CERTIFIED,"ports america group, inc.",chief executives,CHIEF PROCESS OFFICER,Y,138611.0,2016.0,"JERSEY CITY, NEW JERSEY",non university,Management Occupation,JERSEY
3,4,CERTIFIED,"gates corporation, a wholly-owned subsidiary o...",chief executives,"REGIONAL PRESIDEN, AMERICAS",Y,138611.0,2016.0,"DENVER, COLORADO",non university,Management Occupation,COLORADO
5,6,CERTIFIED,burger king corporation,chief executives,"EXECUTIVE V P, GLOBAL DEVELOPMENT AND PRESIDEN...",Y,138611.0,2016.0,"MIAMI, FLORIDA",non university,Management Occupation,FLORIDA


In [56]:
#Convert target variable to binary
from sklearn import preprocessing
class_mapping = {'CERTIFIED':0, 'DENIED':1}
df["CASE_STATUS"] = df["CASE_STATUS"].map(class_mapping)

In [61]:
df['CASE_STATUS'].value_counts()

0    2818282
1      94364
Name: CASE_STATUS, dtype: int64

In [62]:
##Numerical transformation for job title
test1 = pd.Series(df['JOB_TITLE'].ravel()).unique()
print(pd.DataFrame(test1))

                                                        0
0                            POSTDOCTORAL RESEARCH FELLOW
1                                 CHIEF OPERATING OFFICER
2                                   CHIEF PROCESS OFFICER
3                             REGIONAL PRESIDEN, AMERICAS
4       EXECUTIVE V P, GLOBAL DEVELOPMENT AND PRESIDEN...
5                                CHIEF OPERATIONS OFFICER
6                                               PRESIDENT
7          CHIEF INFORMATION OFFICER, HEINZ NORTH AMERICA
8        VICE PRESIDENT AND CHIEF HUMAN RESOURCES OFFICER
9                                       TREASURER AND COO
10                               CHIEF COMMERCIAL OFFICER
11                                           BOARD MEMBER
12                                CHIEF FINANCIAL OFFICER
13                              VICE PRESIDENT OF FINANCE
14                            GENERAL MANAGER, OPERATIONS
15                                                    CEO
16            

In [63]:
# dropping columns
df = df.drop('EMPLOYER_NAME', axis = 1)
df = df.drop('SOC_NAME', axis = 1)
df = df.drop('JOB_TITLE', axis = 1)
df = df.drop('WORKSITE', axis = 1)
df = df.drop('CASE_ID', axis = 1)

In [66]:
df1 = df.copy()

In [67]:
#changing dtype to categorical
df1[['CASE_STATUS', 'FULL_TIME_POSITION', 'YEAR','NEW_EMPLOYER','OCCUPATION','state']] = df1[['CASE_STATUS', 'FULL_TIME_POSITION', 'YEAR','NEW_EMPLOYER','OCCUPATION','state']].apply(lambda x: x.astype('category'))

In [68]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2912646 entries, 0 to 3002392
Data columns (total 7 columns):
CASE_STATUS           category
FULL_TIME_POSITION    category
PREVAILING_WAGE       float64
YEAR                  category
NEW_EMPLOYER          category
OCCUPATION            category
state                 category
dtypes: category(6), float64(1)
memory usage: 61.1 MB


In [69]:
#Split datt into trianing anfd test
X = df.drop('CASE_STATUS', axis=1)
y = df.CASE_STATUS

seed = 7
test_size = 0.40
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


In [70]:
X_train.columns

Index(['FULL_TIME_POSITION', 'PREVAILING_WAGE', 'YEAR', 'NEW_EMPLOYER',
       'OCCUPATION', 'state'],
      dtype='object')

In [71]:
X_train.isnull().sum()

FULL_TIME_POSITION    0
PREVAILING_WAGE       0
YEAR                  0
NEW_EMPLOYER          0
OCCUPATION            0
state                 0
dtype: int64

In [72]:
#Converting it to XGboost 
##pd.get_dummies() is used to encode the categorical values to integers
X_train_encode = pd.get_dummies(X_train)
X_test_encode = pd.get_dummies(X_test)

In [73]:
y_train.head()

1005328    0
816196     0
1582854    0
825095     0
2963078    0
Name: CASE_STATUS, dtype: int64

In [75]:
train_X = X_train_encode.as_matrix()
train_y = y_train.as_matrix()

In [76]:
import xgboost
gbm=xgboost.XGBClassifier(max_features='sqrt', subsample=0.8, random_state=10)

In [77]:
from sklearn.model_selection import GridSearchCV

In [78]:
##set stages and shrinkage
parameters = [{'n_estimators': [10, 100]},
              {'learning_rate': [0.1, 0.01, 0.5]}]

In [79]:
grid_search = GridSearchCV(estimator = gbm, param_grid = parameters, scoring='accuracy', cv = 3, n_jobs=-1)

In [80]:
grid_search = grid_search.fit(train_X, train_y)

In [81]:
grid_search.grid_scores_, grid_search.best_params_, grid_search.best_score_
#training set - Learning rate 0.5, acc = 96.7% 

([mean: 0.96769, std: 0.00000, params: {'n_estimators': 10},
  mean: 0.96770, std: 0.00000, params: {'n_estimators': 100},
  mean: 0.96770, std: 0.00000, params: {'learning_rate': 0.1},
  mean: 0.96769, std: 0.00000, params: {'learning_rate': 0.01},
  mean: 0.96769, std: 0.00003, params: {'learning_rate': 0.5}],
 {'n_estimators': 100},
 0.9676983177375432)

In [82]:
#Best combination of hyperparameters 
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features='sqrt', min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=10, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

In [83]:
#Model fiting
gbm=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=3, max_features='sqrt', min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=10, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8).fit(train_X, train_y)

In [84]:
#prediction on test data set 
y_pred = gbm.predict(X_test_encode.as_matrix())

In [85]:
#Evalation
confusion_matrix(y_test, y_pred)


array([[1127037,     115],
       [  37771,     136]])

In [87]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98   1127152
          1       0.54      0.00      0.01     37907

avg / total       0.95      0.97      0.95   1165059



In [88]:
#Roc curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)
#50% of the area is under the curve

0.5017428504022443